# Get List of Data Sources

In [22]:
import requests, os
from requests.utils import requote_uri
urlString= 'http://api.scholexplorer.openaire.eu/v1/listDatasources'
r = requests.get(urlString)

#Create a URL encoded list of data sources
dataSources=[];
for ds in r.json():
    if len(ds.strip())>0:
        dataSources.append(requote_uri(ds))
        
        
#Print name of datasources
count=0
for ds in dataSources:
    print('{}. {}'.format(count,ds))
    count=count+1
    
print('We found {} data sources.'.format(count))

0. Datasets%20in%20Datacite
1. Cambridge%20Crystallographic%20Data%20Centre
2. European%20Nucleotide%20Archive
3. PubMed
4. Pangaea
5. ICPSR
6. RCSB
7. Elsevier
8. OpenAIRE
9. Springer%20Nature
10. Thomson%20Reuters
11. Australian%20National%20Data%20Service
12. IEDA
13. 3TU.Datacentrum
14. IEEE
15. Mendeley%20Data
We found 16 data sources.


# Download files for a given datasource

In [23]:
SelectedDataSources={0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15}

#Create local folder for the data sources
for i in SelectedDataSources:
    ds=dataSources[i]
    if not os.path.exists(ds):
        print ('Creating new folder: {}'.format(ds))
        os.makedirs(ds)
    else:
        print ('We found a local folder for: {}'.format(ds))

We found a local folder for: Datasets%20in%20Datacite
Creating new folder: Cambridge%20Crystallographic%20Data%20Centre
Creating new folder: European%20Nucleotide%20Archive
Creating new folder: PubMed
Creating new folder: Pangaea
Creating new folder: ICPSR
Creating new folder: RCSB
Creating new folder: Elsevier
Creating new folder: OpenAIRE
We found a local folder for: Springer%20Nature
Creating new folder: Thomson%20Reuters
We found a local folder for: Australian%20National%20Data%20Service
Creating new folder: IEDA
Creating new folder: 3TU.Datacentrum
Creating new folder: IEEE
Creating new folder: Mendeley%20Data


In [24]:
#Download
downloadFiles=True
maximumPages=3

if downloadFiles:
    for i in SelectedDataSources:
        ds=dataSources[i]
        page=0
        statusCode=200
        while (statusCode==200 and page < maximumPages):
            urlString= 'https://api-dliservice-prototype-dli.d4science.org/v1/linksFromDatasource?datasource={}&page={}'.format(ds,page)
            r = requests.get(urlString)
            fileName='{}.json'.format(1000000+page)
            myfile = open('./{}/{}'.format(ds,fileName), 'w')
            myfile.write(r.text)
            statusCode= r.status_code
            page = page + 1
            myfile.close
            if page%10==0:
                print('We have downloaded {} files for {}'.format(page,ds))
    print('Download is complete')
else:
    print('Download is disabled')

Download is complete


# Create CSV from JSON files

## Read Schemas

In [31]:
import glob, json, csv, pprint

nodes={}
relations=[]
schemas=[]

def addToSchemas(identifiers):
    for i in identifiers:
        if i['schema'] not in schemas:
            print(i['schema'])
            schemas.append(i['schema'])

for i in SelectedDataSources:
        ds=dataSources[i]
        path = ds
        dirs = os.listdir(path)
        path = '{}/*.json'.format(ds)
        for fname in glob.glob(path):
            #print(fname)
            data = json.load(open(fname))
            for l in data:
                addToSchemas(l['source']['identifiers'])
                addToSchemas(l['target']['identifiers'])

print('Done!')

doi
dnetIdentifier
url
handle
ena
pmid
icpsr
pdb
pubmedid
openaire
genbank
arrayexpress
geo
nuccore
bind
sra
ddbj
swiss-prot
embl
ensembl
bioproject
pride
mgi
uri
website
local
issn
au-anl:peau
purl
isbn
orcid
uri doi
Done!


## Read JSON files

In [27]:
import glob, json, csv, pprint

nodes={}
relations=[]

def addNode(node,label,fname):
    row={'title':'','label':label,'fname':fname}

    try:
        row['title']=node['title']
    except Exception:
        pass
                
    for i in node['identifiers']:
        if i['schema']=='dnetIdentifier':
            row['local_id']=i['identifier']
        else:
            row[i['schema']]=i['identifier']    

    if row['local_id'] in nodes:
        if 'doi' in row:
            nodes[row['local_id']]=row
    else:
        nodes[row['local_id']]=row     

    return row['local_id']

 


for i in SelectedDataSources:
        ds=dataSources[i]
        path = ds
        dirs = os.listdir(path)
        path = '{}/*.json'.format(ds)
        
        for fname in glob.glob(path):
            print(fname)
            data = json.load(open(fname))
            
            for l in data:                
                label=''
                for p in l['linkProvider']:
                    if label!='':
                        label= label +';'
                    label= label + p['name']

                source_id=addNode(l['source'],label,fname)                                                                 
                target_id=addNode(l['target'],label,fname)

                relationship_type=l['relationship']['name']        
                relation_row=[source_id,target_id,relationship_type]
                if relation_row not in relations:   
                    relations.append(relation_row)

print('Done!')

Datasets%20in%20Datacite/1000004.json
Datasets%20in%20Datacite/1000008.json
Datasets%20in%20Datacite/1000005.json
Datasets%20in%20Datacite/1000002.json
Datasets%20in%20Datacite/1000003.json
Datasets%20in%20Datacite/1000000.json
Datasets%20in%20Datacite/1000001.json
Datasets%20in%20Datacite/1000006.json
Datasets%20in%20Datacite/1000007.json
Cambridge%20Crystallographic%20Data%20Centre/1000002.json
Cambridge%20Crystallographic%20Data%20Centre/1000000.json
Cambridge%20Crystallographic%20Data%20Centre/1000001.json
European%20Nucleotide%20Archive/1000002.json
European%20Nucleotide%20Archive/1000000.json
European%20Nucleotide%20Archive/1000001.json
PubMed/1000002.json
PubMed/1000000.json
PubMed/1000001.json
Pangaea/1000002.json
Pangaea/1000000.json
Pangaea/1000001.json
ICPSR/1000002.json
ICPSR/1000000.json
ICPSR/1000001.json
RCSB/1000002.json
RCSB/1000000.json
RCSB/1000001.json
Elsevier/1000002.json
Elsevier/1000000.json
Elsevier/1000001.json
OpenAIRE/1000002.json
OpenAIRE/1000000.json
OpenA

Springer%20Nature/1000205.json
Springer%20Nature/1002238.json
Springer%20Nature/1008635.json
Springer%20Nature/1005457.json
Springer%20Nature/1002392.json
Springer%20Nature/1001947.json
Springer%20Nature/1000655.json
Springer%20Nature/1002668.json
Springer%20Nature/1007190.json
Springer%20Nature/1008265.json
Springer%20Nature/1002146.json
Springer%20Nature/1005683.json
Springer%20Nature/1004991.json
Springer%20Nature/1007344.json
Springer%20Nature/1000481.json
Springer%20Nature/1005379.json
Springer%20Nature/1002516.json
Springer%20Nature/1007714.json
Springer%20Nature/1005729.json
Springer%20Nature/1006806.json
Springer%20Nature/1004092.json
Springer%20Nature/1003757.json
Springer%20Nature/1002845.json
Springer%20Nature/1000878.json
Springer%20Nature/1001290.json
Springer%20Nature/1004568.json
Springer%20Nature/1006555.json
Springer%20Nature/1008818.json
Springer%20Nature/1003307.json
Springer%20Nature/1004138.json
Springer%20Nature/1006105.json
Springer%20Nature/1006410.json
Springer

Springer%20Nature/1008967.json
Springer%20Nature/1009675.json
Springer%20Nature/1003278.json
Springer%20Nature/1006580.json
Springer%20Nature/1001245.json
Springer%20Nature/1005206.json
Springer%20Nature/1008434.json
Springer%20Nature/1000004.json
Springer%20Nature/1002039.json
Springer%20Nature/1006979.json
Springer%20Nature/1004944.json
Springer%20Nature/1005656.json
Springer%20Nature/1002193.json
Springer%20Nature/1008064.json
Springer%20Nature/1000454.json
Springer%20Nature/1002469.json
Springer%20Nature/1007391.json
Springer%20Nature/1008571.json
Springer%20Nature/1006996.json
Springer%20Nature/1007684.json
Springer%20Nature/1000141.json
Springer%20Nature/1002486.json
Springer%20Nature/1005343.json
Springer%20Nature/1008121.json
Springer%20Nature/1000511.json
Springer%20Nature/1004801.json
Springer%20Nature/1005713.json
Springer%20Nature/1009360.json
Springer%20Nature/1000842.json
Springer%20Nature/1001750.json
Springer%20Nature/1006095.json
Springer%20Nature/1004552.json
Springer

Springer%20Nature/1008855.json
Springer%20Nature/1009747.json
Springer%20Nature/1001377.json
Springer%20Nature/1004525.json
Springer%20Nature/1006518.json
Springer%20Nature/1009317.json
Springer%20Nature/1002808.json
Springer%20Nature/1000835.json
Springer%20Nature/1001727.json
Springer%20Nature/1004876.json
Springer%20Nature/1007759.json
Springer%20Nature/1005764.json
Springer%20Nature/1008156.json
Springer%20Nature/1000566.json
Springer%20Nature/1007309.json
Springer%20Nature/1005334.json
Springer%20Nature/1008506.json
Springer%20Nature/1000136.json
Springer%20Nature/1008013.json
Springer%20Nature/1000423.json
Springer%20Nature/1004933.json
Springer%20Nature/1005621.json
Springer%20Nature/1008443.json
Springer%20Nature/1004899.json
Springer%20Nature/1000073.json
Springer%20Nature/1000589.json
Springer%20Nature/1005271.json
Springer%20Nature/1008910.json
Springer%20Nature/1009602.json
Springer%20Nature/1001232.json
Springer%20Nature/1004030.json
Springer%20Nature/1009252.json
Springer

Springer%20Nature/1004374.json
Springer%20Nature/1006349.json
Springer%20Nature/1001176.json
Springer%20Nature/1009546.json
Springer%20Nature/1005836.json
Springer%20Nature/1004724.json
Springer%20Nature/1006719.json
Springer%20Nature/1001526.json
Springer%20Nature/1009116.json
Springer%20Nature/1007558.json
Springer%20Nature/1005565.json
Springer%20Nature/1001875.json
Springer%20Nature/1003848.json
Springer%20Nature/1000767.json
Springer%20Nature/1008357.json
Springer%20Nature/1007108.json
Springer%20Nature/1005135.json
Springer%20Nature/1000337.json
Springer%20Nature/1009815.json
Springer%20Nature/1008707.json
Springer%20Nature/1006430.json
Springer%20Nature/1003632.json
Springer%20Nature/1002920.json
Springer%20Nature/1006060.json
Springer%20Nature/1003798.json
Springer%20Nature/1009395.json
Springer%20Nature/1003262.json
Springer%20Nature/1007221.json
Springer%20Nature/1002023.json
Springer%20Nature/1007671.json
Springer%20Nature/1002189.json
Springer%20Nature/1006963.json
Springer

...

Australian%20National%20Data%20Service/1008196.json
Australian%20National%20Data%20Service/1007263.json
Australian%20National%20Data%20Service/1007799.json
Australian%20National%20Data%20Service/1002061.json
Australian%20National%20Data%20Service/1007633.json
Australian%20National%20Data%20Service/1006921.json
Australian%20National%20Data%20Service/1002431.json
Australian%20National%20Data%20Service/1009787.json
Australian%20National%20Data%20Service/1008895.json
Australian%20National%20Data%20Service/1006472.json
Australian%20National%20Data%20Service/1006188.json
Australian%20National%20Data%20Service/1003670.json
Australian%20National%20Data%20Service/1002962.json
Australian%20National%20Data%20Service/1006022.json
Australian%20National%20Data%20Service/1003220.json
Australian%20National%20Data%20Service/1003257.json
Australian%20National%20Data%20Service/1004592.json
Australian%20National%20Data%20Service/1008948.json
Australian%20National%20Data%20Service/1006055.json
Australian%2

Australian%20National%20Data%20Service/1000705.json
Australian%20National%20Data%20Service/1003980.json
Australian%20National%20Data%20Service/1002692.json
Australian%20National%20Data%20Service/1005157.json
Australian%20National%20Data%20Service/1009877.json
Australian%20National%20Data%20Service/1008765.json
Australian%20National%20Data%20Service/1007490.json
Australian%20National%20Data%20Service/1002368.json
Australian%20National%20Data%20Service/1000355.json
Australian%20National%20Data%20Service/1004316.json
Australian%20National%20Data%20Service/1009524.json
Australian%20National%20Data%20Service/1001114.json
Australian%20National%20Data%20Service/1003129.json
Australian%20National%20Data%20Service/1007869.json
Australian%20National%20Data%20Service/1005854.json
Australian%20National%20Data%20Service/1004746.json
Australian%20National%20Data%20Service/1003083.json
Australian%20National%20Data%20Service/1009174.json
Australian%20National%20Data%20Service/1001544.json
Australian%2

Australian%20National%20Data%20Service/1002796.json
Australian%20National%20Data%20Service/1005053.json
Australian%20National%20Data%20Service/1003884.json
Australian%20National%20Data%20Service/1005546.json
Australian%20National%20Data%20Service/1002283.json
Australian%20National%20Data%20Service/1000744.json
Australian%20National%20Data%20Service/1007081.json
Australian%20National%20Data%20Service/1002779.json
Australian%20National%20Data%20Service/1001856.json
Australian%20National%20Data%20Service/1008374.json
Australian%20National%20Data%20Service/1005116.json
Australian%20National%20Data%20Service/1000314.json
Australian%20National%20Data%20Service/1002329.json
Australian%20National%20Data%20Service/1008724.json
Australian%20National%20Data%20Service/1009836.json
Australian%20National%20Data%20Service/1003492.json
Australian%20National%20Data%20Service/1004357.json
Australian%20National%20Data%20Service/1006690.json
Australian%20National%20Data%20Service/1003168.json
Australian%2

Australian%20National%20Data%20Service/1005839.json
Australian%20National%20Data%20Service/1007804.json
Australian%20National%20Data%20Service/1009549.json
Australian%20National%20Data%20Service/1003144.json
Australian%20National%20Data%20Service/1001179.json
Australian%20National%20Data%20Service/1004681.json
Australian%20National%20Data%20Service/1005993.json
Australian%20National%20Data%20Service/1006346.json
Australian%20National%20Data%20Service/1001483.json
Australian%20National%20Data%20Service/1008708.json
Australian%20National%20Data%20Service/1000338.json
Australian%20National%20Data%20Service/1002305.json
Australian%20National%20Data%20Service/1007107.json
Australian%20National%20Data%20Service/1008358.json
Australian%20National%20Data%20Service/1000768.json
Australian%20National%20Data%20Service/1005090.json
Australian%20National%20Data%20Service/1002755.json
Australian%20National%20Data%20Service/1003847.json
Australian%20National%20Data%20Service/1000292.json
Australian%2

Australian%20National%20Data%20Service/1000050.json
Australian%20National%20Data%20Service/1006887.json
Australian%20National%20Data%20Service/1002597.json
Australian%20National%20Data%20Service/1005252.json
Australian%20National%20Data%20Service/1001568.json
Australian%20National%20Data%20Service/1004290.json
Australian%20National%20Data%20Service/1003555.json
Australian%20National%20Data%20Service/1009158.json
Australian%20National%20Data%20Service/1007845.json
Australian%20National%20Data%20Service/1005878.json
Australian%20National%20Data%20Service/1001092.json
Australian%20National%20Data%20Service/1006757.json
Australian%20National%20Data%20Service/1001138.json
Australian%20National%20Data%20Service/1003105.json
Australian%20National%20Data%20Service/1009508.json
Australian%20National%20Data%20Service/1006307.json
Australian%20National%20Data%20Service/1002344.json
Australian%20National%20Data%20Service/1000379.json
Australian%20National%20Data%20Service/1005481.json
Australian%2

## Create nodes.csv

In [28]:
nodeFile=open('nodes.csv','w')
nodeWriter = csv.writer(nodeFile, quoting=csv.QUOTE_ALL)
nodeWriter.writerow (['key:ID','title','doi','uri','url','local','handle','pmid','icpsr','pdb','pubmedid','genbank','geo','embl','ensembl','issn','purl','isbn','orcid','json',':LABEL'])

for row in nodes.values():
    doi=row.get('doi','')
    uri=row.get('uri','')
    url=row.get('url','')
    local=row.get('local','') 
    handle=row.get('handle','') 
    pmid=row.get('pmid','') 
    icpsr=row.get('icpsr','') 
    pdb=row.get('pdb','') 
    pubmedid=row.get('pubmedid','')
    genbank=row.get('genbank','')
    geo=row.get('geo','')
    embl=row.get('embl','')
    ensembl=row.get('ensembl','')
    issn=row.get('issn','')
    purl=row.get('purl','')
    isbn=row.get('isbn','')
    orcid=row.get('orcid','')
    nodeWriter.writerow ([row['local_id'],row['title'],doi,uri,url,local,handle,pmid,icpsr,pdb,pubmedid,'genbank','geo','embl','ensembl','issn','purl','isbn','orcid,row['fname'],row['label']])
    
nodeFile.close()
print('Done!')

Done!


## Create relations.csv

In [29]:
nodeFile=open('relations.csv','w')
nodeWriter = csv.writer(nodeFile, quoting=csv.QUOTE_ALL)
nodeWriter.writerow ([':START_ID',':END_ID',':TYPE'])

for row in relations:
    nodeWriter.writerow(row)
    
nodeFile.close()
print('Done!')

Done!
